In [1]:
!pip install emoji
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/carlospenaloza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from textblob import TextBlob
from bs4 import BeautifulSoup
import matplotlib.cm as cm
from string import punctuation
from string import digits
import urllib.request
import networkx as nx
import pandas as pd
import itertools
import emoji
import math
import time
import json
import sys
import re

# TODO: remove this when NetworkX is fixed
from warnings import simplefilter
import matplotlib.cbook
simplefilter("ignore", matplotlib.cbook.mplDeprecation)

In [3]:
# Import tweets
tweets = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/tweets/statuses.csv')

tweets.head()

,id,tweet_id,created_at,text,favorite_count,retweet_count,phone,sensitive,hashtags,no_hashtags,mentions,no_mentions,no_urls,no_media
0,113127283.0,1.172520e+18,9/13/19 14:56,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
1,113127283.0,1.172520e+18,9/13/19 14:54,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,2,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",5,0,1
2,113127283.0,1.172520e+18,9/13/19 14:52,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
3,113127283.0,1.172520e+18,9/13/19 14:50,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
4,113127283.0,1.172520e+18,9/13/19 14:48,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",4,0,1


In [4]:
# Import spanish stop word dictionary
url_sw = 'https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/stopwords-es.json'
response_sw = urllib.request.urlopen(url_sw)
data_sw = response_sw.read()

stop_words = set(json.loads(data_sw))

# Import emoji meanings
emoji_translations = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/emojis_translated.csv')

# Get spanish meaning of an emoji
def emoji_meaning(emoji):
  meaning = emoji_translations.loc[emoji_translations['emoji'] == emoji]['translation']
  return meaning.values[0] if not meaning.empty else ''

# Transformations to remove digits and punctuation
remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans('', '', punctuation)

In [17]:
from ipython_exit import exit

""" Processes text data in tweets
    
    text: text of tweet
    word_counts: word count dictionary
    
    returns processed text, length of content, polarity, subjectivity
"""
def process_text(text, word_counts, user_id):
    # Remove links
    text = ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())
    
    # Remove file names
    
        
    # Remove mentions
    text = ' '.join(re.sub("(@[A-Za-z0-9^\w]+)", " ", text.replace('@ ','@').replace('# ','#')).split())
        
    # Replace hashtags with words
    if text.count('#')>0:
        text = ' '.join(re.findall('[A-Z][^A-Z]*', text.replace('#', ' ')))
            
    #Remove HTML tags
    text = BeautifulSoup(text).get_text()
    
    # Save content length (exluding links and mentions)
    length = len(text)
        
    # Remove punctuation symbols
    text = ' '.join(re.sub("[\.\,\¡\¿\!\?\:\;\-\=\*\(\)\[\]\"\'\“\_\+\”\%\/\‘\’]", " ", text).split())
    text = text.translate(remove_digits).translate(remove_punctuation)
        
    # Lower case to avoid case sensitive problems
    text = text.lower()
        
    # Replace emojis with names 
    text = emoji.demojize(text)
        
    # Add space between emojis and other characters
    ind = -2
    for c in range(text.count(':')):
        ind = text.find(':',ind+2)
        if c%2==0:
            newLetter = ' :'
        else:
            newLetter = ': '
        text ="".join((text[:ind],newLetter,text[ind+1:]))
            
    # Replace emoji names with spanish meaning
    result = []
    parts = text.split(' ')
    for part in parts:
        if part:
            if part[0]==':':
                em = handle_emoji_tone(part)
                em = emoji_meaning(em)
                if em:
                    result.append(em)
            else:
                if part not in stop_words:
                    if part not in word_counts:
                        word_counts[part] = 1
                    else:
                        word_counts[part] += 1
                result.append(part)
        
    text = ' '.join(result)
        
    # Filter using NLTK library append it to a string
    word_tokens = word_tokenize(text)
        
    result = [w for w in word_tokens if not w in stop_words]
        
    text = ' '.join(result)
    
    if len(text)>12:
        analysis = TextBlob(text)
        try:
            eng = analysis.translate(to='en')
            sentiment = eng.sentiment     
            polarity = sentiment.polarity
            subjectivity = sentiment.subjectivity
            time.sleep(.55)
        except Exception as e:
            if str(e)=='HTTP Error 429: Too Many Requests':
                print('Error')
                print('Last user:', user_id)
                exit()
            polarity = 0.0
            subjectivity = 0.0
    
    else:
        polarity = 0.0
        subjectivity = 0.0
    

    return text, length, polarity, subjectivity

tones = ['_light_skin_tone','_medium-light_skin_tone','_medium_skin_tone', 
             '_medium-dark_skin_tone','_dark_skin_tone']

# Method that removes the tone from emojis
def handle_emoji_tone(emoji):  
    for t in tones:
        if t in emoji:
            tone = t
            return emoji.replace(tone,'')
    return emoji


In [9]:
test = '@Tolaymaruja Las esperamos en el cine este fin de semana de estreno de @monosfilm #MONOS las abraza. https://t.co/L5B7xs9M3e '
process_text(test,{},1)

('esperamos cine semana estreno abraza', 81, 0.0, 0.0)

In [7]:
# Import the network
G = nx.read_graphml("network1.graphml")

# List to hold part of the dataset
text_analysis = []
text_analysis.append(['id', 'tweet_id','engagement',
            'effective_length', 'polarity', 'subjectivity'])

# List to store average engagement
avg_eng = []
avg_eng.append(['id', 'average_engagement', 'common_words', 'common_ht_words',
               'common_hts'])

In [ ]:
''' Text analysis '''

last_uid = 269039712
found_start = False

#for n in G.nodes():
for n in list(G.nodes()):
    user_id = int(float(n))
    if not found_start and user_id != last_uid:
        continue
    else:
        found_start = True
    
    print ('User: ', user_id)
    
    # Total engagement
    eng = 0
    tweet_count = 0
    
    # Frequent word dictionary
    freq_words = {}

    # Frequent hashtag dictionary
    freq_hashtags = {}

    # Frequent words in hashtags dictionary
    freq_ht_words = {}
    
    user_tweets = tweets.loc[tweets['id'] == user_id]
    for index, row in user_tweets.iterrows():
      
        # Add current engagement
        currEng = int(row['favorite_count'])+int(row['retweet_count'])
        eng += currEng
        tweet_count +=1
        
        # Get hashtags
        ht = row['hashtags'].split(';') if row['hashtags'] != '-' else []
        
        # Update hashtag count
        for hashtag in ht:
            if hashtag not in freq_hashtags:
                freq_hashtags[hashtag] = 1
            else:
                freq_hashtags[hashtag] += 1
        
        # Separate hashtags by capitalization
        ht_words = [re.findall('[a-zA-Z][^A-Z]*', w) for w in ht]
        
        # Create a list with all the words in the hashtags
        hts = []
        for h in ht_words:
            hts = hts + h
        
        # Remove digits and lower caps for every hashtag word
        hts = [item.translate(remove_digits).lower() for item in hts]
        
        # Update hashtag word frequencies
        for word in hts:
            if word not in stop_words:
                if word not in freq_ht_words:
                    freq_ht_words[word] = 1
                else:
                    freq_ht_words[word] += 1
        
        text, length, polarity, subjectivity = process_text(row['text'], freq_words, user_id)
        
        # Update text in  data frame
        text_analysis.append([n, row['tweet_id'], currEng, length, polarity, subjectivity])
        print('Original:',row['text'],'\nProcessed:',text,'\nPolarity, subjectivity:',
              polarity, subjectivity,'\n')
    
    if tweet_count > 0:
        average_engagement = eng / tweet_count
    else:
        average_engagement = 0
    c_words = Counter(freq_words)
    c_ht_words = Counter(freq_ht_words)
    c_hashtags = Counter(freq_hashtags)
    avg_eng.append([user_id, average_engagement, c_words.most_common(3),
                   c_ht_words.most_common(3), c_hashtags.most_common(3)])

User:  269039712
Original: No esperanza, no te desprestigies de esa forma,  hasta ahi venias muy bien , recuerda que todo lo que toca la politica , lo daña, y tu todo lo tienes muy bien,jajajajajajaj https://t.co/72NEdZhVqy 
Processed: esperanza desprestigies forma venias recuerda toca politica daña jajajajajajaj 
Polarity, subjectivity: 0.0 0.0 

Original: Uyyyyy  pero se ve que usted dominó todo el tiempo, las tenia metidas en el arco  jajajajajaj https://t.co/UZCfr5ZhC5 
Processed: uyyyyy ve dominó tenia metidas arco jajajajajaj 
Polarity, subjectivity: 0.0 0.0 

Original: Con ustedes el show de laura https://t.co/LlGbpQQaJM 
Processed: show laura 
Polarity, subjectivity: 0.0 0.0 

Original: FELICITACIONES  TOLIMITA,  me alegra por mi amigo pitirri salazar, por orlando rojas, por el profe gamero, por chumillo, y por todos mis amigos tolimenses disfrutenlo y ojala salgan campeones https://t.co/tiBETMp63P 
Processed: felicitaciones tolimita alegra amigo pitirri salazar orlando rojas p

Original: Carlitos algo se aprende  , gracias un abrazo https://t.co/qXk8orgu23 
Processed: carlitos aprende gracias abrazo 
Polarity, subjectivity: 0.0 0.0 

Original: Esoooo y a donde le envio las cortesias para usted y su distinguida familia? https://t.co/nYOp24PWHR 
Processed: esoooo envio cortesias distinguida familia 
Polarity, subjectivity: 0.0 0.0 

Original: Jajajajajajaj  nadie fue capaz de avisarle ? https://t.co/ZstjezrI4r 
Processed: jajajajajajaj capaz avisarle 
Polarity, subjectivity: 0.5 0.625 

Original: Es la actriz y cantante mas bella, y linda jajajajaj mi amiga linda lucia callejas esta sobrada.  Es sencilla y gran ser humano https://t.co/SG1BBvECVm 
Processed: actriz cantante bella linda jajajajaj amiga linda lucia callejas sobrada sencilla humano 
Polarity, subjectivity: 0.0 0.2285714285714286 

Original: Para  CARACOL TELEVISION  , solo tengo palabras de Gratitud, agradecimiento, felicitaciones, y  mil adjetivos mas que hoy la posesionan como la mejor productora

Original: LA PAZ DE SANTOS???  Que venga  ponga la cara y diga que hoy es premio nobel por esto. https://t.co/ozobj1cc8P 
Processed: paz santos venga ponga cara diga premio nobel 
Polarity, subjectivity: 0.0 0.0 

Original: Se les dijo a los inocentes y que es gusta montar novelas, es la segunda oarte de su primera llegada al madrid, NO LO VA A PONER,  ( o depronto 15 minutos) y YA. asidesoues dice que contó con el .tercosss https://t.co/f1dpa4BOzf 
Processed: inocentes gusta montar novelas oarte llegada madrid depronto minutos asidesoues contó tercosss 
Polarity, subjectivity: 0.0 0.0 

Original: Que esta bajita una llanta  jajajajaj https://t.co/WcOBc2POJ1 
Processed: bajita llanta jajajajaj 
Polarity, subjectivity: 0.1 0.4 

Original: Es que  no conocieron a zidane? Esas palabras ya las dijo una y mil veces. Escuchen bien , DIJO tengo a TODOSSS, los mejores jugadores,  y cuando le preguntan por james,, REPITE,  todos , estaN, bien y loS pondré, no dijo voy a CONTAR CON JAMES,  que n

Original: Si o no  ? Liz  pa que meter a los muchachos en esto, son solo opiniones https://t.co/hqZakxoJZe 
Processed: liz pa meter muchachos opiniones 
Polarity, subjectivity: 0.0 0.0 

Original: Es su opinion  y al igual que muchas otras las respeto, un abrazo https://t.co/J4d3IBmqmE 
Processed: opinion respeto abrazo 
Polarity, subjectivity: 0.0 0.0 

Original: Cuando aprenda a preguntar y tenga modales hablamos mijo, que igualado ah? https://t.co/zHn55IFvCT 
Processed: aprenda preguntar modales hablamos mijo igualado ah 
Polarity, subjectivity: 0.0 0.0 

Original: Por nada mi sheyla, que Dios te bendiga con vida y salud, para que sigas aportando todo tu talento y forma de ser,por que la belleza esta .,pero no necesitaste de ella para llegar donde estas, felicitaciones y  arriba ocaña y su bolegancho. https://t.co/VrE98sisns 
Processed: sheyla dios bendiga vida salud sigas aportando talento forma belleza necesitaste llegar felicitaciones ocaña bolegancho 
Polarity, subjectivity: 0.0

Original: Barbarita: En Barbaritas Mechas ofrecemos lavado de pelo y limpieza facial. Servicio ⭐⭐⭐⭐⭐ 👌🏽😁 https://t.co/WoPBJZcCxW 
Processed: barbarita barbaritas mechas ofrecemos lavado pelo limpieza facial servicio OK mano 
Polarity, subjectivity: 0.25 0.25 

Original: Hoy en Santa Marta, gran  bingo de obras sociales de la @PoliciaColombia . "Eche culo e calor"  😅🤣😅🤣 @ Santa Marta, Magdalena https://t.co/4wM8Fm1tYM 
Processed: santa marta bingo obras sociales eche culo calor rodando piso riendo rodando piso riendo marta magdalena 
Polarity, subjectivity: 0.03333333333333333 0.06666666666666667 

Original: Yo a NINGUNO, VOTO EN BLANCO https://t.co/OtILW1xvQk 
Processed: voto blanco 
Polarity, subjectivity: 0.0 0.0 

Original: La pregunta es? Po que no los han juzgado? Por que teniendo tantas pruebas como dicen tener, no estan presos? Algo raro pasa no cree? Y santrich? El paisa? Romaña? Pruebas hay , entonces? Xq andan libres? https://t.co/vkX4e9RVDo 
Processed: pregunta po juzgado ta

...

Original: #LasMásLeídas Líderes de los estudiantes de la @udistrital  plantearon ante el Consejo Superior Universitario validar la asamblea universitaria. De su respuesta depende si van a paro o no. https://t.co/WHHe0ZDxIr https://t.co/Gu5GEL0oXV 
Processed: leídas líderes estudiantes plantearon consejo superior universitario validar asamblea universitaria respuesta depende paro 
Polarity, subjectivity: 0.0 0.0 

Original: #VIDEO |  El Rector de la @udistrital , @rgarciaduarte, explicó paso a paso los acontecimientos presentados al interior de la institución frente a las denuncias de corrupción que en este momento son investigadas por los órganos de control. https://t.co/BFnvHHVbXG 
Processed: rector paso paso acontecimientos presentados interior institución frente denuncias corrupción investigadas órganos control 
Polarity, subjectivity: 0.0 0.0 

Original: Escucha los nuevos artistas y propuestas musicales en #Evolución por https://t.co/WfpTKPsPMt https://t.co/8NLk4Df9bc 
Processed: 

Original: Voto por @LaudEstereo en la categoría #MejorMedioDeComunicación de los @PremiosTWCo. Nomina el tuyo en -&gt; https://t.co/nPovQG3fSw 
Processed: voto categoría comunicación nomina 
Polarity, subjectivity: 0.0 0.0 

Original: #AlAire en #AmantesDelCírculoPolar "Un periodista que no lee poesía, no es un periodista" https://t.co/l6LpjZcQTy 
Processed: aire amantes círculo polar periodista lee poesía periodista 
Polarity, subjectivity: -0.08333333333333333 0.25 

Original: #AlAire Las canciones que fueron éxito y se mantienen vivas en #JuevesdeClásicos, un recorrido por los sonidos y las voces que las hacen inmortales. https://t.co/SpwxLTlU3I 
Processed: aire canciones éxito mantienen vivas juevesde clásicos recorrido sonidos voces inmortales 
Polarity, subjectivity: 0.2 0.2 

Original: #VIDEO |  El Rector de la @udistrital , @rgarciaduarte, explicó paso a paso los acontecimientos presentados al interior de la institución frente a las denuncias de corrupción que en este momento s

Original: Lo que fuí, soy y seré... La música siempre salva.  Feliz sábado! A brillar 🌠🌠🌠 #OnStage #Singer #Artist #Music 🎶🎸🎤🎛️🎹🎼 🎥 @oceanlatamph @heyguysmartin https://t.co/R3JCd0KeQs 
Processed: fuí música salva feliz sábado brillar on stage singer artist music guitarra micrófono teclado musical puntuación musical cámara cine 
Polarity, subjectivity: 0.26666666666666666 0.3333333333333333 

Original: Me sigo preguntando.. qué le pusiste al café 🤔☕ https://t.co/YzxZLN8Yjx 
Processed: sigo preguntando pusiste café 
Polarity, subjectivity: 0.0 0.0 

Original: #DesembarcanLosGavilanes  Gracias Florencio Ortiz por tu mágica iluminación, @julianareyes1 por tu bello vestuario, Julián Hoyos por esa impresionante escenografía, Ignacio García y @jalejandroroca por su impecable dirección escénica y musical.  @TuBoletaNews y taquillas https://t.co/Tl07uKOcoa 
Processed: desembarcan gavilanes gracias florencio ortiz mágica iluminación bello vestuario julián hoyos impresionante escenografía ignaci

Original: Empezar la semana cantando siempre será un gran bendición para mí...aunque haya sido bien tempranito 😁 @RCNEspectaculo @NoticiasRCN  #LasCartasSobreLaMesa https://t.co/Gx5zc5ArmM 
Processed: empezar semana cantando bendición tempranito cartas mesa 
Polarity, subjectivity: 0.1 0.3 

Original: Grade @YuriTura!!! Maravilloso espectáculo junto a @SinfonicaNalCol bajo la dirección de Paul Duty. 👏👏👏👏 Sencillamente Exquisito https://t.co/FRVtM9hBns 
Processed: grade maravilloso espectáculo dirección paul duty aplaudir aplaudir aplaudir aplaudir sencillamente exquisito 
Polarity, subjectivity: 1.0 1.0 

Original: Poesía Erótica Para Ser Compartida. @tatirente y éste servidor muy Pronto en el @TeatroColon https://t.co/nBQU6SJaXW 
Processed: poesía erótica compartida servidor 
Polarity, subjectivity: 0.7 0.9 

Original: Visitando a doña Katleya https://t.co/lDhEtuzneM 
Processed: visitando doña katleya 
Polarity, subjectivity: 0.0 0.0 

Original: Y es que nos debíamos por mucho la char

Original: Muchas Gracias!! Quedó muy bonita la entrevista 🙏🙏 https://t.co/EySgICDL3F 
Processed: gracias bonita entrevista 
Polarity, subjectivity: 0.225 0.6 

Original: Nos vemos hoy!!!! https://t.co/EySgICDL3F 
Processed: vemos 
Polarity, subjectivity: 0.0 0.0 

Original: MUCHAS GRACIAS!!!!🙏🙌💪💪 ahí vamos subiendo, gracias por el apoyo https://t.co/06Hg94MegV 
Processed: gracias levantando manos subiendo gracias apoyo 
Polarity, subjectivity: 0.0 0.0 

Original: Escucha mi nueva canción #LasCartasSobreLaMesa por  @DeezerLatam en este link https://t.co/oNdlXnF6aP 
Processed: escucha canción cartas mesa link 
Polarity, subjectivity: 0.0 0.0 

Original: Gracias amigo. Es un gusto para mí saber que te gustó! https://t.co/skEitoJpIf 
Processed: gracias amigo gusto gustó 
Polarity, subjectivity: 0.6 0.8 

Original: Ha llegado la Queen a la House! Y yo feliz😁👑 se nota? Feliz día hoy… https://t.co/zbNnyTISrK 
Processed: llegado queen house feliz corona nota feliz hoy… 
Polarity, subjectivity:

In [18]:
print(len(text_analysis), len(avg_eng))
print(avg_eng[-1])
last_id = str(float('269039712'))
text_analysis = [l for l in text_analysis if not last_id in l]
avg_eng = [l for l in avg_eng if not last_id in l]
print(len(text_analysis), len(avg_eng))

220 4
[167435593, 176.3312101910828, [('aguila', 111), ('liga', 96), ('fecha', 51)], [('aguila', 111), ('liga', 95), ('femenina', 28)], [('LigaAguila', 67), ('LigaAguilaFemenina', 28), ('TorneoAguila', 13)]]
220 4


In [42]:
# Save tweet analysis in a dataframe
analysis_results = pd.DataFrame(text_analysis[1:])
analysis_results.columns = text_analysis[0]
analysis_results["id"] = pd.to_numeric(analysis_results["id"])
analysis_results.to_csv('analysis_results.csv', index=False)
analysis_results.head()

,id,tweet_id,engagement,effective_length,polarity,subjectivity
0,426146744.0,1.142850e+18,42,125,0.600000,0.8
1,426146744.0,1.142570e+18,1441,62,0.000000,0.0
2,426146744.0,1.141490e+18,611,20,0.200000,0.4
3,426146744.0,1.141420e+18,16,184,0.366667,0.5
4,426146744.0,1.141130e+18,180,261,-0.600000,0.9


In [43]:
user_network_data=pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/user_network_data.csv')
user_network_data.head()

,id,katz,betweenness,authority
0,426146744.0,1.580982,0.000668,0.000029
1,162926902.0,3.461433,0.005760,0.000456
2,167435593.0,4.891715,0.003585,0.000369
3,269039712.0,1.375570,0.000193,0.000023
4,62561348.0,2.050953,0.000549,0.000141


In [32]:
# Create vertex data frame
vertices = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/vertices.csv', encoding='utf-8', engine='c')
vertices["id"] = pd.to_numeric(vertices["id"])
vertices = vertices.drop(['description'], axis=1)
vertices.head()

,id,screen_name,followers,friends,verified,created_at,listed,tipo
0,426146744.0,SteevenOrozco,655637,476,False,2011,842,opinion
1,162926902.0,VivaAirCol,210761,30037,False,2010,475,empresa
2,167435593.0,Dimayor,1314384,147,True,2010,1308,competencia
3,269039712.0,cecorrehumor,156975,392,False,2011,183,entretenimiento
4,62561348.0,ReddsColombia,25319,2517,True,2009,62,empresa


In [44]:
# Merge all user info

user_engagement = pd.DataFrame(avg_eng[1:])
user_engagement.columns = avg_eng[0]
user_engagement.head()

user_info = pd.merge(user_network_data, vertices, on='id')
user_info = pd.merge(user_info, user_engagement, on='id')
user_info.head()


,id,katz,betweenness,authority,screen_name,followers,friends,verified,created_at,listed,tipo,average_engagement,common_words,common_ht_words,common_hts
0,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.250000,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG..."
1,162926902.0,3.461433,0.005760,0.000456,VivaAirCol,210761,30037,False,2010,475,empresa,30.823529,"[(viva, 24), (air, 24), (hola, 21)]","[(viva, 3), (air, 3), (rosa, 3)]","[(VivaAirRosa, 3), (MiercolesDeMascotas, 2), (..."
2,167435593.0,4.891715,0.003585,0.000369,Dimayor,1314384,147,True,2010,1308,competencia,176.331210,"[(aguila, 111), (liga, 96), (fecha, 51)]","[(aguila, 111), (liga, 95), (femenina, 28)]","[(LigaAguila, 67), (LigaAguilaFemenina, 28), (..."
3,269039712.0,1.375570,0.000193,0.000023,cecorrehumor,156975,392,False,2011,183,entretenimiento,18.087379,"[(gracias, 33), (abrazo, 17), (amigo, 14)]","[(feliz, 6), (martes, 3), (tbt, 2)]","[(FelizMartes, 3), (felizcumpleañosbogotá, 1),..."
4,62561348.0,2.050953,0.000549,0.000141,ReddsColombia,25319,2517,True,2009,62,empresa,1.066667,"[(amarguras, 58), (perreo, 23), (comparte, 22)]","[(amarguras, 57), (menores, 1), (cerveza, 1)]","[(SinAmarguras, 57), (MenoresSinCerveza, 1)]"


In [45]:
user_info.to_csv('user_info.csv', index=False)

In [46]:
# Merge user and tweet data
processed_data = pd.merge(user_info, analysis_results, on='id')
processed_data.head()

,id,katz,betweenness,authority,screen_name,followers,friends,verified,created_at,listed,tipo,average_engagement,common_words,common_ht_words,common_hts,tweet_id,engagement,effective_length,polarity,subjectivity
0,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.142850e+18,42,125,0.600000,0.800000
1,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.142570e+18,1441,62,0.000000,0.000000
2,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.141490e+18,611,20,0.200000,0.400000
3,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.141420e+18,16,184,0.366667,0.500000
4,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.141130e+18,180,261,-0.600000,0.900000
5,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.140000e+18,39,45,0.300000,0.200000
6,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.138980e+18,16,126,0.416667,0.616667
7,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.138110e+18,61,89,0.000000,0.000000
8,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.135690e+18,20,183,-0.145833,0.875000
9,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.25,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG...",1.130910e+18,14,148,0.383333,0.600000


In [ ]:
tweets = tweets.drop(['text','phone','hashtags','mentions'], axis=1)
processed_data = pd.merge(processed_data, tweets, on='id')
processed_data.head()

In [47]:
# Write processed data to file
processed_data.to_csv('processed_data.csv', index=False)